# Домашнее задание 3

В этом домашнем задании вам предстоит работать с датасетом Quora Question Pairs. Нужно обучить модель, которая по паре вопросов могла бы сказать, являются ли эти вопросы дубликатами друг друга или нет. В датасете представлены пары вопросов и разметка для них в формате столбца isDuplicate (true если пара - дубликаты друг друга, false иначе).

Разбалловка за ДЗ следующая:



*   Предобработка и токенизатор - 1 балл
*   Загрузка и обучение модели - 2 балла
*   Пайплайн - 1 балл
*   Результаты - 5 баллов
*   Отчёт - 1 балл
*   Бонусное задание - 5 баллов

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install evaluate --quiet

Note: you may need to restart the kernel to use updated packages.


In [7]:
import torch
import evaluate
from datasets import load_dataset, ClassLabel, Value, load_from_disk
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification, AutoModel
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import DataCollatorWithPadding
from datasets import load_metric

import pandas as pd
import numpy as np
# Все импорты лучше кладите сюда, а не раскидывайте по ячейкам

## Предобработка и токенизатор (1 балл)

Загрузим наш датасет. Совершим определенные преобразования и разобьем выборку на обучающую и тестовую. По тестовой выборке будут оцениваться результаты, поэтому не меняйте ничего в коде, который ее создает.

In [ ]:
ds = load_dataset('quora')["train"]

Для дальнейшей работы нам нужно воспользоваться преобразовать наши данные в формат, который поймет наша модель. Каждую пару предложений нам нужно склеить в одну последовательность, начинающуюся с токена CLS, между которыми будет токен SEP и в конце которой будет стоять токен SEP. Кроме этого, всю разметку нужно перевести в тип ClassLabel вместо текущего бинарного и перенести в признак с названием "labels".

Если хорошо покопаться в документации библиотеки tokenizers, то это все можно сделать довольно легко с помощью аргументов при создании и применении токенизатора.

Разбиение на обучающую и тестовую выборку нужно только для нашей проверки, если хотите создать валидационную выборку для подбора гиперпараметров, такая возможность есть.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

In [ ]:
def tokenize_function(example):
    questions = example['questions']
    t1 = []
    t2 = []
    for t in questions:
        t1.append(t['text'][0])
        t2.append(t['text'][1])
    return tokenizer(t1, t2, truncation=True)

In [ ]:
tokenized_datasets = ds.map(tokenize_function, batched=True)

In [ ]:
td = tokenized_datasets.rename_column('is_duplicate', 'labels')
td = td.class_encode_column('labels')
td = td.remove_columns('questions')

In [ ]:
assert "input_ids" in td.features
assert "labels" in td.features
assert type(td.features['labels']) == ClassLabel

In [ ]:
# DO NOT CHANGE ANYTHING HERE
td = td.train_test_split(test_size=0.2, stratify_by_column='labels', seed=42)
assert len(td["test"]) == 80858

In [4]:
# чтоб не генерировать каждый раз
td = load_from_disk('/content/drive/MyDrive/NNLP_HW3/nnlp_hw3_tdata3')

## Обучение модели (2 балла)

В выборе модели у вас относительная свобода - можете выбирать любую из моделей, основанных на слое Encoder трансформера. Можете использовать предобученные модели, замораживать веса, добавлять слои - все, на что хватит воображения и что даст скор получше.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
model = AutoModelForSequenceClassification.from_pretrained('roberta-base', num_labels=2)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [21]:
def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [20]:
training_args = TrainingArguments(output_dir="/content/drive/MyDrive/NNLP_HW3/q_roberta_right_test",
                                  evaluation_strategy="epoch",
                                  per_device_train_batch_size = 32,
                                  per_device_eval_batch_size = 32,
                                  save_strategy = 'epoch',
                                  num_train_epochs=2,
                                  save_total_limit=1)

trainer = Trainer(model=model, 
                  args=training_args,
                  train_dataset=td['train'],
                  eval_dataset=td['test'],
                  compute_metrics=compute_metrics,
                 data_collator=data_collator)

In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning

  warnings.warn(

***** Running training *****

  Num examples = 323432

  Num Epochs = 2

  Instantaneous batch size per device = 32

  Total train batch size (w. parallel, distributed & accumulation) = 32

  Gradient Accumulation steps = 1

  Total optimization steps = 20216

  Number of trainable parameters = 124647170

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.270800,0.266900,0.884402,0.853864


***** Running Evaluation *****

  Num examples = 80858

  Batch size = 32

<ipython-input-6-38a3d51cb394>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate

  metric = load_metric("glue", "mrpc")


Saving model checkpoint to /content/drive/MyDrive/NNLP_HW3/q_roberta_right_test/checkpoint-10108

Configuration saved in /content/drive/MyDrive/NNLP_HW3/q_roberta_right_test/checkpoint-10108/config.json

Model weights saved in /content/drive/MyDrive/NNLP_HW3/q_roberta_right_test/checkpoint-10108/pytorch_model.bin


## Создание пайплайна (1 балл)

Здесь вам нужно создать пайплайн, который мог бы принять на вход два отдельных предложения в виде строк и выдать ответ True, если эти два предложения являются дублирующимися вопросами и False иначе.

In [5]:
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/NNLP_HW3/q_roberta_right_test/checkpoint-20216', num_labels=2)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import Pipeline

In [ ]:
class PairClassification(Pipeline):
    def _sanitize_parameters(self, **kwargs):
        preprocess_kwargs = {}
        if "second_text" in kwargs:
            preprocess_kwargs["second_text"] = kwargs["second_text"]
        return preprocess_kwargs, {}, {}

    def preprocess(self, text, second_text=None):
        return self.tokenizer.encode(text, text_pair=second_text, return_tensors=self.framework)

    def _forward(self, model_inputs):
        return self.model(model_inputs)

    def postprocess(self, model_outputs):
        return (model_outputs.logits[0][0] < model_outputs.logits[0][1]).item()


In [ ]:
pipeline = PairClassification(model=model, tokenizer=tokenizer, framework='pt', device=0)

In [ ]:
assert type(pipeline(("What's the best way to learn English?",
                     "How can I learn to speak fluent English?"))) == bool

## Отчёт (1 балл)

Здесь в свободной форме можно составить отчет о процессе и результатах работы. Вопросы для вдохновения: 

*   Что пробовали сделать?
*   Что получилось хорошо?
*   Что не получилось сделать? Почему?
*   Что можно было бы еще попробовать? Как улучшить результат?

Текущий результат после файн-тюнинга сносный, но:\
(1) Хотелось бы попробовать что-то, что не энкодер, а энкодер-декодер типа T5 или XLNet, потому что по моему опыту, полные трансформеры справляются со сложными штуками получше\
(2) Еще у меня была авангардная мысль затестить свежерелизнутый FlexGen и попробовать вытащить few-shot'ом что-то из большой OPT (я не думаю, что это улучшит результаты, но мне интересно....)\
(3) У меня не получилось переустновить встроенный в каггл datasets из-за проблем с pyarrow (я через три часа выяснила, что проблема не во мне, но все равно грустно (https://www.kaggle.com/discussions/product-feedback/325765)), а дефолтный имеет версию 2.1.0 и не дает сделать stratify_by_column, чтобы получить тест как в тетрадке. Поэтому мне пришлось отказаться от идеи поставить что-то large учиться 8+ часов на маленьком батчсайзе (лимиты в колабе такое не разрешают)

## Проверка результатов (5 баллов).

Проверим качество получившейся модели. Чтобы получить оценку 5, достаточно просто сделать модель, которая будет работать и что-то выдавать. Чтобы получить баллы за результаты, нужно набрать logloss на тестовой выборке ниже 0.25. Чтобы получить 5 баллов, нужно набрать logloss не выше 0.15. Любое значение logloss между 0.15 и 0.25 оценивается пропорционально (points = (0.25 - your_score) * 50).

In [21]:
trainer.evaluate()

***** Running Evaluation *****

  Num examples = 80858

  Batch size = 32

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


<ipython-input-19-38a3d51cb394>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate

  metric = load_metric("glue", "mrpc")


{'eval_loss': 0.24016359448432922,
 'eval_accuracy': 0.9044497761507828,
 'eval_f1': 0.8728397906448534,
 'eval_runtime': 323.2677,
 'eval_samples_per_second': 250.127,
 'eval_steps_per_second': 7.817}

In [2]:
## TODO: calculate log-loss on df['test']

score = 0.24016359448432922
mark = min([(0.25 - score) * 50, 5])
mark

0.4918202757835388

## Бонус - использование Сиамских сетей (5 баллов)*

Для задач определения похожести пар (особенно пар изображений) нередко используют архитектуру Сиамских сетей. Ее суть заключается в том, что к каждому из двух входов модели применяются две идентичные подмодели с общим набором весов. Результаты применения этих подмоделей к двум предложениям сравниваются с помощью определенной метрики, которую считаем расстоянием. Итоговая модель стремится сделать так, чтобы расстояния между полученными векторами предложений одного класса были макисмально близки, а расстояние между векторами предложений разных классов - максимально далеки. Реализуется это с помощью, например, Triplet Loss или Contrastive Loss.

Подробнее о Сиамских сетях можно почитать [здесь](https://www.projectpro.io/article/siamese-neural-networks/718#mcetoc_1gke10clo18).

Ваша задача - реализовать Сиамскую сеть для нашей задачи определения схожих вопросов и сравнить результаты ее работы с результатами работы обычного трансформера.

Можете использовать любые инструменты и библиотеки, главное - результат.

**Я вынесла это в отдельную [тетрадку](https://drive.google.com/file/d/1yWjmM7Zx9NK9PxLM2FAbaoaUk6wtrAoW/view?usp=sharing)**